In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import csv
import math
import operator

In [16]:
df_demographic = pd.read_csv('hh_demographic.csv')

In [18]:
print(len(df_demographic['household_key'].unique()))

801


In [6]:
print(df_demographic['AGE_DESC'].unique())

['65+' '45-54' '25-34' '35-44' '19-24' '55-64']


In [7]:
len(df_demographic)

801

In [8]:
len(df_demographic['household_key'].unique())

801

In [9]:
print(df_demographic['INCOME_DESC'].unique())

['35-49K' '50-74K' '25-34K' '75-99K' 'Under 15K' '100-124K' '15-24K'
 '125-149K' '150-174K' '250K+' '175-199K' '200-249K']


In [10]:
print(df_demographic['HOMEOWNER_DESC'].unique())

['Homeowner' 'Unknown' 'Renter' 'Probable Renter' 'Probable Owner']


In [11]:
print(df_demographic['KID_CATEGORY_DESC'].unique())

['None/Unknown' '1' '2' '3+']


In [12]:
df_transaction = pd.read_csv('transaction_data.csv')
print(df_transaction.head())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  
0       364        -0.60        1631        1          0.0                0.0  
1       364         0.00        1631        1          0.0                0.0  
2       364        -0.30        1631        1          0.0                0.0  
3       364         0.00        1631        1          0.0                0.0  
4       364        -0.39        1631        1          0.0                0.0  


In [74]:
df_coup_red = pd.read_csv('coupon_redempt.csv', dtype={'COUPON_UPC': str})

In [21]:
len(df_coup_red)

2318

In [22]:
coupon_hh = sum(df_coup_red.duplicated(['COUPON_UPC']))
print(coupon_hh)

1762


In [25]:
df_coup_red_dup = df_coup_red[df_coup_red.duplicated(['COUPON_UPC'])].sort_values(['COUPON_UPC'])

In [26]:
print(df_coup_red_dup.head(20))

      household_key  DAY   COUPON_UPC  CAMPAIGN
572             588  348  10000085320        30
411             389  442  10000085361         8
598             605  427  10000085361         8
1430           1708  425  10000085361         8
331             306  420  10000085361         8
319             294  424  10000085361         8
1421           1693  455  10000085361         8
186             117  426  10000085361         8
305             276  437  10000085361         8
1407           1677  425  10000085361         8
543             574  417  10000085361         8
2192           2430  424  10000085361         8
272             256  429  10000085361         8
1690           1965  418  10000085361         8
258             243  417  10000085361         8
1126           1337  416  10000085361         8
528             545  459  10000085361         8
1132           1356  421  10000085361         8
2215           2451  415  10000085361         8
1596           1864  430  10000085361   

In [6]:
coupon_count = df_coup_red.groupby(["CAMPAIGN"]).agg('count')

In [8]:
coupon_count

,household_key,DAY,COUPON_UPC
CAMPAIGN,,,
1,1,1,1
2,5,5,5
3,2,2,2
4,11,11,11
5,13,13,13
6,1,1,1
7,7,7,7
8,372,372,372
9,43,43,43


In [9]:
camp_18_coupons = df_coup_red[df_coup_red.CAMPAIGN == 18]

In [10]:
len(camp_18_coupons)

653

In [11]:
camp_18_coupons.head()

,household_key,DAY,COUPON_UPC,CAMPAIGN
3,1,597,10000085476,18
4,1,597,54200029176,18
17,13,609,10000089277,18
18,13,609,51800015050,18
19,13,609,53500035050,18


In [13]:
camp_18_coupons_list = camp_18_coupons['COUPON_UPC'].unique()
print(camp_18_coupons_list)

[10000085476 54200029176 10000089277 51800015050 53500035050 54105800093
 53980010082 10000085475 55000013950 52500010050 10000085484 10000089284
 51600070033 51800010250 54300026050 54850010033 57410101082 52000030042
 54060060076 54133500050 54154888076 10000089290 51906341075 53010010033
 57631000076 57218040076 54900050076 10000085477 10000085479 51960010076
 52100023024 52840001082 55410000076 10000089257 51111120001 51570011042
 57294010033 10000089248 51600030101 57680000050 53800050033 51920021576
 10000089266 53700026850 57332150033 10000085478 52200090076 57143020076
 53160401082 51200000076 52113100033 10000089251 51116200076 57124960082
 56687861776 53700013182 10000089250 54800111033 10000089244 10000089272
 55480017133 50581447025 10000089240 10000089285 10000089252 55703711082
 51862700076 52800051098 10000089306 10000089271 57144800076 53120026033
 10000089263 10000089270 10000089239 52570016075 50004510076 55400061078
 10000089246 10000089253 10000089288 51111114101 53

In [14]:
df_transactions = pd.read_csv('transaction_data.csv')

In [15]:
print(df_transactions.head())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  
0       364        -0.60        1631        1          0.0                0.0  
1       364         0.00        1631        1          0.0                0.0  
2       364        -0.30        1631        1          0.0                0.0  
3       364         0.00        1631        1          0.0                0.0  
4       364        -0.39        1631        1          0.0                0.0  


In [127]:
df_coup = pd.read_csv('coupon.csv', dtype={'COUPON_UPC': str, 'CAMPAIGN': str, 'PRODUCT_ID': str})
#len(df_coup[df_coup['COUPON_UPC'].apply(lambda x: True if str(x) == '10000085364' else False)])


In [128]:
df_coup_red = pd.read_csv('coupon_redempt.csv', dtype={'COUPON_UPC': str, 'household_key': str, 'DAY': str, 'CAMPAIGN': str})

In [129]:
df_merged_coup_red_coup = df_coup.merge(df_coup_red, on=['COUPON_UPC', 'CAMPAIGN'], how='left')

In [143]:
#df_merged_coup_red_coup.head()
df1 = df_merged_coup_red_coup[df_merged_coup_red_coup['household_key'] == '208']
#df1 = df1[df1['CAMPAIGN'] == '22']
len(df1)

4486

In [146]:
len(df_coup_red[df_coup_red['household_key'] == '208'])
#df_coup.head()

6

In [131]:
df_transaction = pd.read_csv('transaction_data.csv', dtype={'BASKET_ID': str, 'PRODUCT_ID': str, 'household_key': str, 'DAY': str})

In [132]:
df_transaction.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [141]:
df_john = df_transaction[(df_transaction['household_key'] == '208') & (df_transaction['COUPON_DISC'] != 0.0)]

In [142]:
len(df_john)

166

In [119]:
df_merged_coup_red_coup_transaction = df_transaction.merge(df_merged_coup_red_coup, on=['household_key', 'DAY', 'PRODUCT_ID'], how='left')

In [126]:
df_merged_coup_red_coup_transaction[(df_merged_coup_red_coup_transaction['COUPON_UPC'].isnull() == False) & (df_merged_coup_red_coup_transaction['household_key'] == '208')]

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,COUPON_UPC,CAMPAIGN
1444279,208,33293475744,427,1004895,2,4.58,327,-1.20,1237,62,-1.0,0.0,55100090033,8
2120842,208,40666652921,597,1017772,1,0.88,327,-0.61,2039,86,-0.5,-0.5,51800015050,18
2120855,208,40666652921,597,15596461,1,2.24,327,-0.75,2039,86,-1.0,0.0,51920021576,18
2138131,208,40715247809,601,896292,2,4.00,327,-2.58,1914,87,-1.0,0.0,55410000076,18
2156873,208,40765530992,606,845715,1,1.99,327,0.00,1206,87,0.0,0.0,10000085475,18
2156878,208,40765530992,606,901062,1,3.80,327,0.00,1206,87,0.0,0.0,10000085475,18
2156888,208,40765530992,606,1037863,1,0.94,327,0.00,1206,87,0.0,0.0,10000085475,18
2156890,208,40765530992,606,1055646,1,0.99,327,0.00,1206,87,0.0,0.0,10000085475,18
2156893,208,40765530992,606,1068715,1,1.29,327,-0.60,1206,87,0.0,0.0,10000085475,18
2156894,208,40765530992,606,1117514,1,2.50,327,0.00,1206,87,0.0,0.0,10000085475,18


In [123]:
len(df_merged_coup_red_coup_transaction)

2595914

In [124]:
df_merged_coup_red_coup_transaction.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,COUPON_UPC,CAMPAIGN
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,NaN


In [ ]:
print(df_transaction[df_transaction.household_key == "208"])